In [ ]:
# Chains in LangChain

## Connect to OpenAI and define our model

Create an OpenAI API key for this project at [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys). 

Copy `.env.sample` to `.env` and replace `<YOUR_OPENAI_API_KEY>` with the one you created.

Let's get started!

Let's make sure that we can read our OpenAI key from the `.env` file we created.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Specify our model
llm_model="gpt-3.5-turbo-1106"

## Load our example CSV file using pandas

Before we begin, let's install the following dependencies from the command-line of our Python virtual environment:

```sh
(.venv) % pip install pandas
(.venv) % pip freeze > requirements.txt
```

In [5]:
import pandas as pd
df = pd.read_csv('../data/Data.csv')

VERIFY: Make sure we have some data loaded into our dataframe

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# Initialize the language model we want to use with a high temperature to have creative description
llm = ChatOpenAI(temperature=0.9, model=llm_model)

/Users/rob/repos/deeplearning-ai/langchain-for-llm-application-development/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Our prompt
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
# Combine the LLM and the prompt into a chain
chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
# Use the chain to come up with a name for a company that makes queen size sheet sets
product = "Queen Size Sheet Set"
chain.run(product)

'"Regal Linens"'

Experiment with different products or re-run the chain to see what responses get generated.

This will work with products that do not exist in the source CSV file. Give it a shot. 🤓

## SimpleSequentialChain

SimpleSequentialChain's are useful when you need to serially execute chains - typically chains that have one input and one output.

In the example below:

- Our first chain will take in the product name defined above - "Queen Size Sheet Set"
  - The output will be a creative name for a company to make the product - queen size sheet sets in our case
- Our second chain will receive the creative name for a company and write a creative twenty word description for the company name

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [17]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Dreams Linens"
Royal Dreams Linens offers luxurious and elegant bedding and home decor, providing customers with the ultimate in comfort and style.

> Finished chain.


'Royal Dreams Linens offers luxurious and elegant bedding and home decor, providing customers with the ultimate in comfort and style.'

OK, but what about cases where there are multiple inputs and/or outputs? For that, we will need to use a `SequentialChain`

## SequentialChain

When there are multiple inputs and/or outputs, we will need to use a `SequentialChain`

In [20]:
from langchain.chains import SequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate the review to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [22]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary of the previously generated English review
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [23]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [24]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# We are using the output_key of chain 2 - summary - and the output_key of chain 3 - language - as inputs to chain 4

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [25]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [26]:
# Choose a review from our dataframe
review = df.Review[5]

# Pass the review to the overall chain
overall_chain(review)

/Users/rob/repos/deeplearning-ai/langchain-for-llm-application-development/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the mediocre taste and lack of foam in the product, suggesting it may be an old batch or counterfeit.',
 'followup_message': "Cher client,\nNous sommes désolés d'apprendre que vous n'êtes pas satisfait de votre expérience avec notre produit. Nous comprenons votre déception et nous souhaitons résoudre ce problème au plus vite. Nous prendrons les mesures nécessaires pour enquêter sur la qualité du lot en question. N'hésitez pas à nous contacter directement pour que nous puissions rectifier cette situation. Nous vous remercions pour votre honnêteté et votre soutien à notre marque.\nCordialement,\nL

## Router Chain